In [ ]:

import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy
import numpy as np
import cv2

#import tensorflow as tf
import hashlib
import pathlib

from PIL import Image
from six import BytesIO



In [ ]:
os.chdir('/content/drive/MyDrive/Wobot.ai')


In [ ]:
base_dir = '/content/drive/MyDrive/Wobot.ai/HardHat_Dataset'

In [ ]:
train_dir = os.path.join(base_dir, 'train')


In [ ]:
os.mkdir(train_dir)

In [ ]:
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

In [ ]:
from shutil import copyfile

In [ ]:
all_imgs = os.listdir(os.path.join(base_dir, 'images'))
random.seed(1)
random.shuffle(all_imgs)

train_imgs = all_imgs[:3800]
test_imgs = all_imgs[3800:]

In [ ]:
for i in range(len(train_imgs)):
  original_path = os.path.join(os.path.join(base_dir, 'images'), train_imgs[i])
  new_path = os.path.join(train_dir, train_imgs[i]) 
  copyfile(original_path, new_path)


In [ ]:
for i in range(len(test_imgs)):
  original_path = os.path.join(os.path.join(base_dir, 'images'), test_imgs[i])
  new_path = os.path.join(test_dir, test_imgs[i])
  copyfile(original_path, new_path)

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
def read_content_from_xml(xml_file: str):
  tree = ET.parse(xml_file)
  root = tree.getroot()

  list_with_all_boxes = []

  for boxes in root.iter('object'):
    
    filename = root.find('filename').text
    classname = boxes.find("name").text
    
    ymin, xmin, ymax, xmax = None, None, None, None
      
    ymin = int(float(boxes.find("bndbox/ymin").text))
    xmin = int(float(boxes.find("bndbox/xmin").text))
    ymax = int(float(boxes.find("bndbox/ymax").text))
    xmax = int(float(boxes.find("bndbox/xmax").text))

    list_with_Separate_boxes = [filename, xmin, ymin, xmax, ymax, classname]
    list_with_all_boxes.append(list_with_Separate_boxes)
  return list_with_all_boxes

In [ ]:
annot_dir = os.path.join(base_dir, 'annotations')

In [ ]:
import pandas as pd

In [ ]:
train_annot = pd.DataFrame()
for i in train_imgs:
  annot_file = i[:-3] + 'xml'
  row = read_content_from_xml(os.path.join(annot_dir, annot_file))
  if row:
    train_annot = train_annot.append(row, ignore_index = True)
train_annot.columns = ['name', 'xmin', 'ymin', 'xmax', 'ymax', 'class']


In [ ]:
train_annot.to_csv('/content/drive/MyDrive/Wobot.ai/HardHat_Dataset/train_labels.csv',index=None)

In [ ]:
test_annot = pd.DataFrame()

for i in test_imgs:
  annot_file = i[:-3] + 'xml'
  row = read_content_from_xml(os.path.join(annot_dir, annot_file))
  if row:
    test_annot = test_annot.append(row, ignore_index = True)
test_annot.columns = ['name', 'xmin', 'ymin', 'xmax', 'ymax', 'class']

In [ ]:
test_annot.to_csv('/content/drive/MyDrive/Wobot.ai/HardHat_Dataset/test_labels.csv',index= None)

In [ ]:
def convert_classes(classes, start=1):
    msg = ''
    for id, name in enumerate(classes, start=start):
        msg = msg + "item {\n"
        msg = msg + " id: " + str(id) + "\n"
        msg = msg + " name: '" + name + "'\n}\n\n"
    return msg[:-1]

In [ ]:
label_map = convert_classes(['helmet', 'head', 'person'])
with open("tf_label_map.pbtxt", "w") as f:
    f.write(label_map)
    f.close()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Wobot.ai/test_labels.csv')

In [ ]:
df2 = df.rename(columns={'name':'filename'})

In [ ]:
df2.to_csv('/content/drive/MyDrive/Wobot.ai/models/research/object_detection/data/test_labels.csv',index=None)

In [ ]:
df3 = pd.read_csv('/content/drive/MyDrive/Wobot.ai/train_labels.csv')

In [ ]:
df4 = df3.rename(columns={'name':'filename'})

In [ ]:
df4.to_csv('/content/drive/MyDrive/Wobot.ai/models/research/object_detection/data/train_labels.csv',index=None)